In [60]:
import pandas as pd
import datetime as dt

In [61]:
path = 'D:/Data Analytics Bootamp/Week 13 - ETL (Extract, Transform, Load)/145153_1116148_upload_stocks_latest'

#Create our dataframes
dividends_df = pd.read_csv(f'{path}/dividends_latest.csv')
earnings_df = pd.read_csv(f'{path}/earnings_latest.csv')
prices_df = pd.read_csv(f'{path}/stock_prices_latest.csv')

In [62]:
# Convert the pandas object to a datetime object
dividends_df['date'] = pd.to_datetime(dividends_df['date'])
earnings_df['date'] = pd.to_datetime(earnings_df['date'])
prices_df['date'] = pd.to_datetime(prices_df['date'])

In [63]:
# Create our 'year' column in order to group by
dividends_df['year'] = pd.DatetimeIndex(dividends_df['date']).year
earnings_df['year'] = pd.DatetimeIndex(earnings_df['date']).year
prices_df['year'] = pd.DatetimeIndex(prices_df['date']).year

In [64]:
# Create a 'differential' column, difference between close and open price
prices_df['differential'] = prices_df['close'] - prices_df['open']
prices_df.head()

,symbol,date,open,high,low,close,year,differential
0,MSFT,2016-05-16,50.80,51.96,50.75,51.83,2016,1.03
1,MSFT,2002-01-16,68.85,69.84,67.85,67.87,2002,-0.98
2,MSFT,2001-09-18,53.41,55.00,53.17,54.32,2001,0.91
3,MSFT,2007-10-26,36.01,36.03,34.56,35.03,2007,-0.98
4,MSFT,2014-06-27,41.61,42.29,41.51,42.25,2014,0.64


In [65]:
# Drop the NAs from our datasets
dividends_df = dividends_df.dropna()
earnings_df = earnings_df.dropna()
prices_df = prices_df.dropna()

In [66]:
# Group our datasets and generate our aggregate statistics
#g_dividends_df = dividends_df.groupby('symbol_year').agg({'dividend': ['mean', 'min', 'max','sum']})
g_dividends_df = dividends_df.groupby(['symbol', 'year']).agg({'dividend': ['mean', 'min', 'max','sum']})
g_dividends_df.columns = ['dividend_mean', 'dividend_min', 'dividend_max','dividend_sum']
g_dividends_df = g_dividends_df.reset_index()
g_dividends_df['symbol_year'] = g_dividends_df['symbol'].astype(str) + g_dividends_df['year'].astype(str)

g_earnings_df = earnings_df.groupby(['symbol', 'year']).agg({'eps': ['sum']})
g_earnings_df.columns = ['eps_sum']
g_earnings_df = g_earnings_df.reset_index()
g_earnings_df['symbol_year'] = g_earnings_df['symbol'].astype(str) + g_earnings_df['year'].astype(str)

g_prices_df = prices_df.groupby(['symbol', 'year']).agg({'low':'min','high':'max','differential':'mean'})
g_prices_df.columns = ['low_min','high_max','avg_differential']
g_prices_df = g_prices_df.reset_index()
g_prices_df['symbol_year'] = g_prices_df['symbol'].astype(str) + g_prices_df['year'].astype(str)

In [67]:
g_dividends_df.head()

,symbol,year,dividend_mean,dividend_min,dividend_max,dividend_sum,symbol_year
0,A,2012,0.10000,0.100,0.100,0.400,A2012
1,A,2013,0.12300,0.120,0.132,0.492,A2013
2,A,2014,4.03650,0.132,15.750,16.146,A2014
3,A,2015,0.10300,0.100,0.115,0.515,A2015
4,A,2016,0.11925,0.115,0.132,0.477,A2016


In [68]:
g_earnings_df.head()

,symbol,year,eps_sum,symbol_year
0,A,2012,0.84,A2012
1,A,2013,2.89,A2013
2,A,2014,2.33,A2014
3,A,2015,1.73,A2015
4,A,2016,1.98,A2016


In [69]:
g_prices_df.head()

,symbol,year,low_min,high_max,avg_differential,symbol_year
0,AAAP,2015,18.00,32.92,0.287003,AAAP2015
1,AAAP,2016,20.53,39.66,0.024980,AAAP2016
2,AAAP,2017,26.60,81.75,0.173925,AAAP2017
3,AAAP,2018,81.61,82.74,0.046875,AAAP2018
4,AABA,2000,25.06,500.10,-0.713016,AABA2000


In [75]:
# Import SQL Alchemy
from sqlalchemy import create_engine
# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [82]:
# Create our classes
class Dividends(Base):
    __tablename__ = 'dividends'
    symbol_year = Column(String(20),primary_key=True)
    symbol = Column(String(10))
    year = Column(String(4))
    dividend_mean = Column(Integer)
    dividend_min = Column(Integer)
    dividend_max = Column(Integer)
    dividend_sum = Column(Integer)

class Earnings(Base):
    __tablename__ = 'earnings'
    symbol_year = Column(String(20),primary_key=True)
    symbol = Column(String(10))
    year = Column(String(4))
    eps_sum = Column(Integer)

class Prices(Base):
    __tablename__ = 'prices'
    symbol_year = Column(String(20),primary_key=True)
    symbol = Column(String(10))
    year = Column(String(4))
    low_min = Column(Integer)
    high_max = Column(Integer)
    avg_differential = Column(Integer)

In [83]:
# Create Database Connection
# ----------------------------------
# Establish Connection
engine = create_engine("sqlite:///stonks.sqlite")
conn = engine.connect()

In [84]:
# Create both the Surfer and Board tables within the database
Base.metadata.create_all(conn)

In [85]:
# To push the objects made and query the server we use a Session object
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [86]:
g_dividends_df.to_sql("dividends",engine,if_exists='replace')
g_earnings_df.to_sql("earnings",engine,if_exists='replace')
g_prices_df.to_sql("prices",engine,if_exists='replace')